In [1]:
from dscribe.descriptors import SOAP
from ase import Atoms
import numpy as np

In [2]:
#import dataset 

import os
import pandas as pd

# === Paths ===
BASE_DIR = 'molecular-property-prediction-challenge'
DIPOLE_FILE = os.path.join(BASE_DIR, 'dipole_moments_train.csv')
STRUCTURE_DIR = os.path.join(BASE_DIR, 'structures_train')  #for test can change dir later

# === Load dipole moment labels ===
dipole_df = pd.read_csv(DIPOLE_FILE)

# === Function to load .xyz file and return DataFrame ===
def parse_xyz(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()
    
    num_atoms = int(lines[0].strip())
    atom_lines = lines[2:2 + num_atoms]
    
    data = []
    for line in atom_lines:
        parts = line.strip().split()
        atom = parts[0]
        x, y, z = map(float, parts[1:])
        data.append((atom, x, y, z))
        
    return pd.DataFrame(data, columns=['atom', 'x', 'y', 'z'])

# === Load all structure files into a single DataFrame ===
def load_structures(structure_dir):
    all_data = []
    
    for filename in os.listdir(structure_dir):
        if filename.endswith('.xyz'):
            mol_name = filename.replace('.xyz', '')
            filepath = os.path.join(structure_dir, filename)
            df = parse_xyz(filepath)
            df['molecule_name'] = mol_name
            df['atom_index'] = range(len(df))  #keeping track of atom index, this is not informative 
            all_data.append(df)
    
    structures_df = pd.concat(all_data, ignore_index=True)
    return structures_df

# === Load training structure data ===
train_structures = load_structures(STRUCTURE_DIR)

# === Merge dipole moment target with training structures ===
train_df = train_structures.merge(dipole_df, on='molecule_name')

In [3]:
# === For test, structure loading only ===
def load_test_structures(test_structure_dir):
    return load_structures(test_structure_dir)
#get test here 
STRUCTURE_DIR_TEST = os.path.join(BASE_DIR, 'structures_test')  #for test can change dir later

test_structures = load_test_structures(STRUCTURE_DIR_TEST)

In [8]:
unique_atoms = sorted(train_df['atom'].unique().tolist())
unique_atoms 

['C', 'F', 'H', 'N', 'O']

# Soap

In [4]:
def xyz_df_to_ase(df):
    symbols = df['atom'].tolist()
    positions = df[['x', 'y', 'z']].values
    return Atoms(symbols=symbols, positions=positions)

In [ ]:
from dscribe.descriptors import SOAP

species = unique_atoms  # as we discussed earlier
soap = SOAP(
    species=species,
    periodic=False,
    r_cut=5.0,     # 5.0 Å captures typical atomic interactions well
    n_max=8,       # good resolution of radial distribution
    l_max=6,       # decent angular resolution
    sparse=False   # set to True if memory becomes an issue
)

In [ ]:
soap_vectors = []
mol_names = []

for mol_name, group in train_df.groupby('molecule_name'):
    mol_atoms = xyz_df_to_ase(group)
    soap_vec = soap.create(mol_atoms)  # shape: (n_atoms, n_features)
    
    # You can aggregate per-molecule: e.g., mean pooling
    soap_mean = soap_vec.mean(axis=0)
    
    soap_vectors.append(soap_mean)
    mol_names.append(mol_name)

# Final dataset
X_soap = np.array(soap_vectors)
y_soap = dipole_df.set_index('molecule_name').loc[mol_names]['dipole_moment'].values

In [16]:
X_soap.shape

(20000, 5740)

In [20]:
from sklearn.decomposition import PCA

# Let's reduce to 100 dimensions (you can tune this later)
pca = PCA(n_components=1000)

# Fit PCA on your entire SOAP data (or just X_train)
X_soap_reduced = pca.fit_transform(X_soap)

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

X_train, X_val, y_train, y_val = train_test_split(X_soap_reduced, y_soap, test_size=0.2, random_state=42)

In [22]:
model = RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)

preds = model.predict(X_val)
mae = mean_absolute_error(y_val, preds)
print(f"MAE (SOAP descriptors): {mae:.4f}")

MAE (SOAP descriptors): 0.7256
